In [ ]:
import pandas as pd

# Proceso ETL

### Extracción de los datos

In [ ]:
df = pd.read_csv('Datos_proyecto_II_BI_2017.csv',encoding='latin-1')
df2 = pd.read_csv('Datos_proyecto_II_BI_2021.csv',encoding='latin-1')

### Transformación 

In [ ]:
df.rename(columns={'LOCALIDAD_TEX':'Localidad','COD_UPZ':'UPZ_id','CODLOCALIDAD':'Localidad_id','NPCEP17':'Cult_id'},inplace=True)
df2.rename(columns={'NOMBRE_LOCALIDAD':'Localidad','COD_UPZ_GRUPO':'UPZ_id','COD_LOCALIDAD':'Localidad_id','NPCEP17':'Cult_id'},inplace=True)
df['year_id']=1
df2['year_id']=2
frames=[df,df2]

In [ ]:
def funct(a):
    try:
        return a.upper()
    except:
        return a

In [ ]:
localiz = pd.concat(frames)
localiz['Localidad'] = localiz.apply(lambda a : funct(a['Localidad']),axis=1)
localiz['Localidad'] = localiz.apply(lambda a : 'ANTONIO NARIÑO' if a['Localidad']== 'ANTONIO NARIÃ\x91O' else a['Localidad'],axis=1)
localiz['Localidad'] = localiz.apply(lambda a : 'LA CANDELARIA' if a['Localidad']== 'CANDELARIA' else a['Localidad'],axis=1)

localzDimension = pd.DataFrame(localiz[['UPZ_id','Localidad','Localidad_id']].value_counts(dropna=True).reset_index(name='count')).drop(columns=['count'])
localzDimension.to_csv('dimensionLocalizacion.csv', index=False)
## Se define UPZ_id como llave primaria / hay 3219 registros vacios en 'UPZ_id' que se eliminan 


In [ ]:
year =  {
    'year_id':[1,2],
    'year':[2017,2021]
}

yearDimension = pd.DataFrame.from_dict(year)
yearDimension
yearDimension.to_csv('dimensionAño.csv', index=False)

In [ ]:
cult =  {
    'Cult_id':[1,2,3,4,5,6],
    'CulturaEtnia':['Indígena','Gitano','Raizal del archipiélago de San Andrés Providencia y Santa Catalina',
        'Palenquero(a) de San Basilio','Negro(a) mulato(a) (afrodescendiente)','Ninguno de los anteriores']
}

culturDimension = pd.DataFrame.from_dict(cult)
culturDimension
culturDimension.to_csv('dimensionCultural.csv', index=False)

In [ ]:
gastSalud = {
    'gasto_id':['NPCFP25BA','NPCFP27','NPCFP33AA','NPCFP33BA','NPCFP33CA','NPCFP33DA','NPCFP33EA','NPCFP33FA','NPCFP34','NPCFP35AA','NPCFP35BA'],
    'descripcion' : ['Gasto mensual en cuidador','Gasto en hospitalizaciones en el ultimo año ','Gasto en consulta de medicina general ultimos 30 días','Gasto en consulta con especialista ultimos 30 días','Gasto en consulta o tratamiento odontológico ultimos 30 días','Gasto en vacunas ultimos 30 días','Gasto en  Laboratorio clínico RX exámenes de diagnóstico ultimos 30 días','Gasto en terapias alternativas ultimos 30 días','Gasto en medicamentos ultimos 30 días ','Gastos  Lentes audífonos o aparatos ortopédicos ultimos 12 meses','Gastos Cirugías o procedimientos quirúrgicos ambulatorios ultimos 12 meses']
}

gastSaludDimension = pd.DataFrame.from_dict(gastSalud)
gastSaludDimension.to_csv('dimensionGastoSalud.csv', index=False)

In [ ]:
factTable = pd.concat(frames).drop(columns=['Localidad','Localidad_id'])
factTable = factTable.dropna(subset=['UPZ_id'])
fact_table_=pd.DataFrame(columns=['UPZ_id','Cult_id','year_id','gasto_id','valor'])
for i in gastSalud['gasto_id']:
    aux = factTable[['UPZ_id','Cult_id','year_id',i]].copy()
    aux['gasto_id']=i
    aux[i] = aux[i].fillna(0)
    aux.rename(columns={i:'valor'},inplace=True)
    fact_table_=pd.concat([fact_table_,aux])
factTable=fact_table_
factTable.to_csv('factTable.csv', index=False)

### Carga 
Se cargan los archivos .csv a la db

In [ ]:
!pip install psycopg2

In [ ]:
import csv
import psycopg2

conn = psycopg2.connect("host= localhost port=5432  dbname=BI user= postgres password= postgres")

cur = conn.cursor()

def cargarSQL(filename, cursor, conex):
    tabla = filename.replace(".csv", "")
    with open(filename, 'r') as f:
        next(f) # Skip the header row.
        cursor.copy_from(f, tabla.lower(), sep=',')
    conex.commit()

#Tabla dimensionLocalizacion 
filename = "dimensionLocalizacion.csv"
fileInput = open(filename, "r")
tabla = filename.replace(".csv", "")
firstLine = fileInput.readline().strip()
columns = firstLine.split(",")
sqlQueryCreate = 'DROP TABLE IF EXISTS '+ tabla + " CASCADE;\n"
sqlQueryCreate += 'CREATE TABLE '+ tabla + "("
sqlQueryCreate += "UPZ_id VARCHAR(64) PRIMARY KEY,\n"
sqlQueryCreate += "Localidad VARCHAR(64),\n"
sqlQueryCreate += "Localidad_id VARCHAR(64)\n"
sqlQueryCreate += ");"
cur.execute(sqlQueryCreate)
conn.commit()
cargarSQL(filename,cur,conn)


#Tabla dimensionCultural 
filename = "dimensionCultural.csv"
fileInput = open(filename, "r")
tabla = filename.replace(".csv", "")
firstLine = fileInput.readline().strip()
columns = firstLine.split(",")
sqlQueryCreate = 'DROP TABLE IF EXISTS '+ tabla + " CASCADE;\n"
sqlQueryCreate += 'CREATE TABLE '+ tabla + "("
sqlQueryCreate += "Cult_id VARCHAR(64) PRIMARY KEY,\n"
sqlQueryCreate += "CulturaEtnia VARCHAR(70)\n"
sqlQueryCreate += ");"
cur.execute(sqlQueryCreate)
conn.commit()
cargarSQL(filename,cur,conn)

#Tabla dimensionCultural 
filename = "dimensionCultural.csv"
fileInput = open(filename, "r")
tabla = filename.replace(".csv", "")
firstLine = fileInput.readline().strip()
columns = firstLine.split(",")
sqlQueryCreate = 'DROP TABLE IF EXISTS '+ tabla + " CASCADE;\n"
sqlQueryCreate += 'CREATE TABLE '+ tabla + "("
sqlQueryCreate += "Cult_id VARCHAR(64) PRIMARY KEY,\n"
sqlQueryCreate += "CulturaEtnia VARCHAR(70)\n"
sqlQueryCreate += ");"
cur.execute(sqlQueryCreate)
conn.commit()
cargarSQL(filename,cur,conn)
        
    
#Tabla dimensionGastoSalud
filename = "dimensionGastoSalud.csv"
fileInput = open(filename, "r")
tabla = filename.replace(".csv", "")
firstLine = fileInput.readline().strip()
columns = firstLine.split(",")
sqlQueryCreate = 'DROP TABLE IF EXISTS '+ tabla + " CASCADE;\n"
sqlQueryCreate += 'CREATE TABLE '+ tabla + "("
sqlQueryCreate += "gasto_id VARCHAR(64) PRIMARY KEY,\n"
sqlQueryCreate += "descripcion VARCHAR(80)\n"
sqlQueryCreate += ");"
cur.execute(sqlQueryCreate)
conn.commit()
cargarSQL(filename,cur,conn)
    
#Tabla dimensionAño
filename = "dimensionAño.csv"
fileInput = open(filename, "r")
tabla = filename.replace(".csv", "")
firstLine = fileInput.readline().strip()
columns = firstLine.split(",")
sqlQueryCreate = 'DROP TABLE IF EXISTS '+ tabla  + " CASCADE;\n"
sqlQueryCreate += 'CREATE TABLE '+ tabla  + "("
sqlQueryCreate += "year_id INTEGER PRIMARY KEY,\n"
sqlQueryCreate += "year INTEGER\n"
sqlQueryCreate += ");"
cur.execute(sqlQueryCreate)
conn.commit()
cargarSQL(filename,cur,conn)


#Tabla de hechos 
filename = "factTable.csv"
fileInput = open(filename, "r")
tabla = filename.replace(".csv", "")
firstLine = fileInput.readline().strip()
columns = firstLine.split(",")
sqlQueryCreate = 'DROP TABLE IF EXISTS '+ tabla + ";\n"
sqlQueryCreate += 'CREATE TABLE '+ tabla + "("
sqlQueryCreate += "UPZ_id VARCHAR(64) references dimensionlocalizacion(upz_id),\n"
sqlQueryCreate += "Cult_id VARCHAR(64) references dimensioncultural(cult_id),\n"
sqlQueryCreate += "year_id INTEGER references dimensionaño(year_id),\n"
sqlQueryCreate += "gasto_id VARCHAR(64) references dimensiongastosalud(gasto_id),\n"
sqlQueryCreate += "valor real \n"
sqlQueryCreate += ");"
cur.execute(sqlQueryCreate)
conn.commit()
cargarSQL(filename,cur,conn)
       
cur.close()